<a href="https://colab.research.google.com/github/anilaksu/Algorithmic-Trading-Codes/blob/TankX-Real-Time-Arbitrage-Case/RealTimeArbitrage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TankX Real-Time Arbitrage Case**

Anil Aksu    

*February 19th, 2024*  

## Notebook Organization:

*   **Housekeeping**
*   **Pair identification**
*   **Data Preparation**
  * Pulling trading data for the pairs form a triangle $A/B-B/C-A/C$
  * Data grouping and reduction
*   **Finding triangular arbitrage opportunities**
  * Checking arbitrage opportunities at each second
  * Ranking and identifying top 5 arbitrage opportunities
*  **Saving all arbitrage opportunities**

#1.Housekeeping

*   Set the working directory
*   Install WebSocket Library
*   Import all relevant libraries



In [1]:
# Here we set our working directory in our google drive to access top 500 pairs with the highest volume and save the outputs
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/ColabNotebooks/TankX Real Time Arbitrage Case
!ls # special shell command to view the files in the home directory of the notebook environment

Mounted at /content/gdrive
/content/gdrive/MyDrive/ColabNotebooks/TankX Real Time Arbitrage Case
ask_arbitrage_oppotunity.csv  bid_arbitrage_oppotunity.csv  BinanceTradingPairs.xlsx
AskDataRaw.csv		      BidDataRaw.csv		    RealTimeArbitrage.ipynb
AskDataRaw.gsheet	      BinanceTradingPairs.gsheet


In [2]:
# Here we setup websocket-client
!python3 -V
!which pip3
!pip3 install websocket-client --upgrade --no-cache-dir

Python 3.10.12
/usr/local/bin/pip3


In [5]:
# Here we import relevant packages
import pandas as pd                 # Pandas library for data organization and manupulation
import numpy as np
import matplotlib.pyplot as plt
import collections                  # List operations
from datetime import datetime       # Importing datetime module
import pytz                         # pytz allows us to set timezone
import json                         # Json library helps us to retrieve online json formatted data
from itertools import permutations  # Allows us to check all potential triangular arbitrage opportunities for given list of trading pairs
import websocket

#2.Pair identification

*   Import top 500 pairs with the largest trading volume
*   Reduce the list of pairs to ones which form a triangle



In [6]:
def getTriangularArbitrageList(splitted_pairs):
  '''
    This function identifies all triangular arbitrage permutations by
    checking if 3 pairs form a set with size 3
  '''
  # Here we form all possible triangular arbitrage permutations
  tri_all_permutations = permutations([x for x in range(0, len(splitted_pairs))], 3)
  # List of an actual triangular pairs by index
  tri_arbitrage_list = []
  Nonexistant = True # Flag to indicate a permutation already exists in the triangular arbitrage list

  for permutation in tri_all_permutations:
    # Here order the permutation to make sure we don't add it more than one with different combination
    permutation_ordered = [list(permutation)[0], list(permutation)[1], list(permutation)[2]]
    # Here we form a set and check if it has 3 elements
    assets_permutation = [splitted_pairs[permutation_ordered[0]],
                              splitted_pairs[permutation_ordered[1]],splitted_pairs[permutation_ordered[2]]]
    all_assets = set().union(*assets_permutation)

    if len(all_assets) == 3:
      # Here we make sure the order forms a cycle such that [A/B, B/C, A/C]
      if(splitted_pairs[permutation_ordered[0]][1] == splitted_pairs[permutation_ordered[2]][0]):
        permutation_ordered = [list(permutation)[0], list(permutation)[2], list(permutation)[1]]
      elif(splitted_pairs[permutation_ordered[1]][1] == splitted_pairs[permutation_ordered[2]][0]):
        permutation_ordered = [list(permutation)[1], list(permutation)[2], list(permutation)[0]]
      elif(splitted_pairs[permutation_ordered[1]][1] == splitted_pairs[permutation_ordered[0]][0]):
        permutation_ordered = [list(permutation)[1], list(permutation)[0], list(permutation)[2]]
      elif(splitted_pairs[permutation_ordered[2]][1] == splitted_pairs[permutation_ordered[0]][0]):
        permutation_ordered = [list(permutation)[2], list(permutation)[0], list(permutation)[1]]
      elif(splitted_pairs[permutation_ordered[2]][1] == splitted_pairs[permutation_ordered[1]][0]):
        permutation_ordered = [list(permutation)[2], list(permutation)[1], list(permutation)[0]]
      else:
        continue # Here the order is correct and we leave it as it is

      # Here we check if a permutation already exists in the list
      for permutation in tri_arbitrage_list:
        if collections.Counter(permutation) == collections.Counter(permutation_ordered) :
          Nonexistant = False
        else:
          Nonexistant = True

      if Nonexistant:
        tri_arbitrage_list.append(permutation_ordered)

  return tri_arbitrage_list

In [7]:
# Top 500 Binance Trading Pairs based on their trading volume
binance_pairs = pd.read_excel('BinanceTradingPairs.xlsx', sheet_name= 'Final Pairs')['Binance Trading Pairs'].tolist()
# Here we split pairs to form a list of triangular arbitrage cycles
splitted_pairs = [s.split("/") for s in binance_pairs]
# We get all triangular arbitrage pairs
tri_arbitrage_list = getTriangularArbitrageList(splitted_pairs)

In [8]:
# Here we remove / in trading pairs because naming convention in WebSockets
binance_pairs = [s.replace('/', '') for s in binance_pairs]
# We form a list of triangular arbitrage to check real time and eliminate pairs which do not form a triangular arbitrage opportunity
triangular_pairs = [[binance_pairs[tri[0]], binance_pairs[tri[1]],
                     binance_pairs[tri[2]]] for tri in tri_arbitrage_list ]
# to convert lists to dictionary with splitted pairs to check if any triangular arbitrage opportunity shows later using getTriangularArbitrageList() function
pairs_dict = {binance_pairs[i]: splitted_pairs[i] for i in range(len(binance_pairs))}
# Here we reduce our list pairs to the ones forming an triangular arbitrage opportunity
pairs_refined = list(set().union(*triangular_pairs))
# Here we refine the pairs dictionary to the ones forming an triangular arbitrage opportunity
pairs_dict = {key: value for key, value in pairs_dict.items() if key in pairs_refined}
print("Number of pairs we track is: ", len(pairs_dict))

Number of pairs we track is:  244


#3.Data Preperation

##3.1 Getting real-time trade data using WebSockets

In [10]:
# Here we organize our merge our pairs and form socket name
all_pairs = [coin.lower() + '@bookTicker' for coin in pairs_refined] # Formatted list
all_pairs= '/'.join(all_pairs)                         # Joined list
socket = "wss://stream.binance.us:9443/ws/"+all_pairs
socket

'wss://stream.binance.us:9443/ws/ntrntry@bookTicker/maticfdusd@bookTicker/mboxusdt@bookTicker/btceur@bookTicker/injusdt@bookTicker/stxbtc@bookTicker/skltry@bookTicker/aptbtc@bookTicker/lunctry@bookTicker/tiausdt@bookTicker/aceusdt@bookTicker/memeusdt@bookTicker/fdusdtry@bookTicker/apttry@bookTicker/imxusdt@bookTicker/rndrusdt@bookTicker/databtc@bookTicker/ethfdusd@bookTicker/injusdc@bookTicker/trxusdt@bookTicker/ustcusdt@bookTicker/avaxbtc@bookTicker/wldusdt@bookTicker/levertry@bookTicker/bnbeth@bookTicker/apeusdt@bookTicker/chzusdt@bookTicker/xaifdusd@bookTicker/seitry@bookTicker/mavtry@bookTicker/opbtc@bookTicker/xmrusdt@bookTicker/dogeusdt@bookTicker/nearusdt@bookTicker/stxtry@bookTicker/dogetry@bookTicker/mantabtc@bookTicker/trbusdt@bookTicker/reeftry@bookTicker/linkfdusd@bookTicker/solusdt@bookTicker/bonkfdusd@bookTicker/stxusdt@bookTicker/atomusdt@bookTicker/arbfdusd@bookTicker/dymtry@bookTicker/usdttry@bookTicker/btcusdc@bookTicker/shibtry@bookTicker/eurusdt@bookTicker/mboxtry@b

**Individual Symbol Book Ticker Streams**

**Change `ticker` to `pair` in data frames!!!**

**Payload:**

* `u`: order book updateId
* `s`: symbol
* `b`: best bid price
* `B`: best bid qty
* `a`: best ask price
* `A`: best ask qty

In [3]:
def manipulation(source):
  '''
    This function organizes messages from WebSocket and fills into a dataframe: bid_data, ask_data
  '''
  ticker = source['s']
  bid_price =source['b']
  ask_price =source['a']
  # Here we keep exact time and time up to second for further analysis
  event_time = datetime.now(pytz.timezone('Asia/Istanbul'))
  # Bid Data
  bid_data = pd.DataFrame(bid_price, columns = ['bid_price'], index = [event_time])
  bid_data['ticker'] = ticker
  bid_data.index.name = 'timestamp'
  bid_data['bid_price'] = bid_data['bid_price'].astype(float)
  bid_data = bid_data.reset_index()
  # Ask data
  ask_data = pd.DataFrame(bid_price, columns = ['ask_price'], index = [event_time])
  ask_data['ticker'] = ticker
  ask_data.index.name = 'timestamp'
  ask_data['ask_price'] = ask_data['ask_price'].astype(float)
  ask_data = ask_data.reset_index()
  # Merged Data for data pulling updates
  df = pd.merge(bid_data, ask_data, left_on=['timestamp','ticker'], right_on=['timestamp','ticker'], how='left')
  df = df[['timestamp', 'ticker', 'bid_price', 'ask_price']] # Here we reorder colums for convenience of the output
  print(df)                                                  # Here we output data for intermediate checks

  return bid_data, ask_data

In [11]:
df_bid = pd.DataFrame()
df_ask = pd.DataFrame()

def on_message(ws, message):
  message = json.loads(message)
  bid_data, ask_data = manipulation(message)
  global df_bid, df_ask                     # Here we access df dataframe globally
  df_bid = pd.concat([df_bid ,   bid_data]) # Here we update bid dataframe
  df_ask = pd.concat([df_ask ,   ask_data]) # Here we update bid dataframe

def on_error(ws, error):
  print(error)

def on_close(ws, close_status_code, close_msg):
  print("### closed ###")
  # Time Formatting for bid data
  df_bid['timestamp'] = pd.to_datetime(df_bid['timestamp'], unit = 'ms')
  df_bid['event_second'] = pd.to_datetime(df_bid['timestamp'].dt.strftime("%m/%d/%Y, %H:%M:%S"))
  # Time Formatting for bid data
  df_ask['timestamp'] = pd.to_datetime(df_ask['timestamp'], unit = 'ms')
  df_ask['event_second'] = pd.to_datetime(df_ask['timestamp'].dt.strftime("%m/%d/%Y, %H:%M:%S"))
  print(df_bid, df_ask)

def on_open(ws):
  print("Opened connection")

ws = websocket.WebSocketApp(socket,
                            on_open = on_open,
                            on_message = on_message,
                            on_error = on_error,
                            on_close = on_close)

In [232]:
ws.run_forever()

Opened connection
                         timestamp   ticker  bid_price  ask_price
0 2024-02-17 16:51:27.666399+03:00  BCHUSDT      263.7      263.7
                         timestamp   ticker  bid_price  ask_price
0 2024-02-17 16:51:27.780854+03:00  BCHUSDT      263.7      263.7
                         timestamp   ticker  bid_price  ask_price
0 2024-02-17 16:51:27.865393+03:00  SOLUSDC     106.89     106.89
                         timestamp   ticker  bid_price  ask_price
0 2024-02-17 16:51:27.882030+03:00  SOLUSDC     106.89     106.89
                         timestamp   ticker  bid_price  ask_price
0 2024-02-17 16:51:27.901285+03:00  LTCUSDT      69.26      69.26
                         timestamp   ticker  bid_price  ask_price
0 2024-02-17 16:51:27.917486+03:00  LTCUSDT      69.26      69.26
                         timestamp    ticker  bid_price  ask_price
0 2024-02-17 16:51:27.929601+03:00  LINKUSDT     19.558     19.558
                         timestamp   ticker  bid_price  

True

**Here I write the raw data as csv file, which inludes data pulled for 2 and half hours. If you want to test my data, you can skip the following cell**

In [235]:
# Here we write the raw data
bid_file = 'BidDataRaw.csv'
ask_file = 'AskDataRaw.csv'
# saving the excel
df_bid.to_csv(bid_file)
df_ask.to_csv(ask_file)

**Here we read the raw data generated on 2/17/2024, if you just want to focus on the analyses, you can simply use this data and continue**

In [12]:
# Here we read the previously saved data
df_bid = pd.read_csv('BidDataRaw.csv')[['timestamp',	'bid_price',	'ticker',	'event_second']]
df_ask = pd.read_csv('AskDataRaw.csv')[['timestamp',	'ask_price',	'ticker',	'event_second']]

##3.1 Data grouping and reduction
Here, we group bid and ask data by each second and pair, then we pick the latest entry for bid and ask price at each second.


In [15]:
#Here we group by event second and get the latest update on prices in each second
df_bid_latest = df_bid.groupby(['event_second', 'ticker']).nth(-1).drop('timestamp', axis = 1).reset_index()
df_ask_latest = df_ask.groupby(['event_second', 'ticker']).nth(-1).drop('timestamp', axis = 1).reset_index()

#4.Analysis of Time Series for identifying triangular arbitrage opportunities

In [13]:
def getTriangularArbitrageOpportunities(df_bid_latest, df_ask_latest, pairs_dict, time):
  '''
    This function checks arbitrage opportunities and stores in a dataframe
  '''
  pairs = df_bid_latest['ticker'].tolist()                            # Here we get trading pairs at each second
  splitted_pairs = [pairs_dict[pair] for pair in pairs]  # Here we get splitted pairs to check which combination has arbitrage potential
  tri_arbitrage_list = getTriangularArbitrageList(splitted_pairs)     # We get all triangular arbitrage pairs
  # We form a list of triangular arbitrage for bid
  triangular_pairs = [[pairs[tri[0]], pairs[tri[1]], pairs[tri[2]]] for tri in tri_arbitrage_list ]
  triangular_bid = [[arb[0], df_bid_latest[df_bid_latest['ticker'] == arb[0]]['bid_price'].item(),
                     arb[1], df_bid_latest[df_bid_latest['ticker'] == arb[1]]['bid_price'].item(),
                     arb[2], df_bid_latest[df_bid_latest['ticker'] == arb[2]]['bid_price'].item()] for arb in triangular_pairs ]
  # We form a list of triangular arbitrage for ask
  triangular_ask = [[arb[0], df_bid_latest[df_ask_latest['ticker'] == arb[0]]['bid_price'].item(),
                     arb[1], df_bid_latest[df_ask_latest['ticker'] == arb[1]]['bid_price'].item(),
                     arb[2], df_bid_latest[df_ask_latest['ticker'] == arb[2]]['bid_price'].item()] for arb in triangular_pairs ]

  # Here we form a dataframe with arbitrage opportunities for bid
  bid_arbitrage = pd.DataFrame(triangular_bid, columns = ['pair_1', 'bid_1' , 'pair_2', 'bid_2', 'pair_3', 'bid_3'])
  bid_arbitrage['event_second'] = time
  # Here we calculate the ratio to check if there is an arbitrage opportunity
  bid_arbitrage['ratio'] = bid_arbitrage['bid_1'] * bid_arbitrage['bid_2'] / bid_arbitrage['bid_3']
  # Here we reorder colums for convenience of the output
  bid_arbitrage = bid_arbitrage[['event_second', 'pair_1', 'bid_1' , 'pair_2', 'bid_2', 'pair_3', 'bid_3', 'ratio']]
  # set the index to the 'event_second' column
  bid_arbitrage.set_index('event_second', inplace=True)

  # Here we form a dataframe with arbitrage opportunities for bid
  ask_arbitrage = pd.DataFrame(triangular_ask, columns = ['pair_1', 'ask_1' , 'pair_2', 'ask_2', 'pair_3', 'ask_3'])
  ask_arbitrage['event_second'] = time
  # Here we calculate the ratio to check if there is an arbitrage opportunity
  ask_arbitrage['ratio'] = ask_arbitrage['ask_1'] * ask_arbitrage['ask_2'] / ask_arbitrage['ask_3']
  # Here we reorder colums for convenience of the output
  ask_arbitrage = ask_arbitrage[['event_second', 'pair_1', 'ask_1' , 'pair_2', 'ask_2', 'pair_3', 'ask_3', 'ratio']]
  # set the index to the 'event_second' column
  ask_arbitrage.set_index('event_second', inplace=True)

  global df_bid_arbitrage, df_ask_arbitrage
  df_bid_arbitrage = pd.concat([df_bid_arbitrage ,   bid_arbitrage]) # Here we update bid dataframe
  df_ask_arbitrage = pd.concat([df_ask_arbitrage ,   ask_arbitrage]) # Here we update ask dataframe


##4.1 Checking arbitrage at each second

In [16]:
# Here we create list of event second to check arbitrage opportunities for each second
event_second = df_bid_latest['event_second'].drop_duplicates().tolist()

df_bid_arbitrage = pd.DataFrame()
df_ask_arbitrage = pd.DataFrame()

for time in event_second:
  getTriangularArbitrageOpportunities(df_bid_latest[df_bid_latest['event_second'] == time],
                                      df_ask_latest[df_ask_latest['event_second'] == time], pairs_dict, time)

# Here we do housekeeping of data tables
df_bid_arbitrage.drop_duplicates(inplace = True)
df_ask_arbitrage.drop_duplicates(inplace = True)
df_bid_arbitrage.reset_index(inplace=True)
df_ask_arbitrage.reset_index(inplace=True)

##4.2 Ranking and identifying top 5 arbitrage opportunities

In [17]:
# Top 5 arbitrage opportunities for ask prices
df_ask_arbitrage.sort_values(by = 'ratio', ascending = True).head(5)

,event_second,pair_1,ask_1,pair_2,ask_2,pair_3,ask_3,ratio
1244,2024-02-17 15:03:46,LINKBTC,0.000370,BTCUSDT,51675.10,LINKUSDT,19.491,0.980955
1290,2024-02-17 15:05:35,LINKBTC,0.000370,BTCUSDT,51674.82,LINKUSDT,19.490,0.981000
1283,2024-02-17 15:05:26,LINKBTC,0.000370,BTCUSDT,51675.10,LINKUSDT,19.490,0.981005
988,2024-02-17 14:46:36,LINKBTC,0.000371,BTCUSDT,51703.14,LINKUSDT,19.517,0.982829
698,2024-02-17 14:34:40,LINKBTC,0.000370,BTCUSDT,51732.86,LINKUSDT,19.487,0.983315


In [18]:
# Top 5 arbitrage opportunities for bid prices
df_bid_arbitrage.sort_values(by = 'ratio', ascending = False).head(5)

,event_second,pair_1,bid_1,pair_2,bid_2,pair_3,bid_3,ratio
2875,2024-02-17 16:20:53,SOLETH,0.03911,ETHUSDT,2766.50000,SOLUSDT,107.43000,1.007147
2888,2024-02-17 16:21:01,SOLETH,0.03905,ETHUSDC,2765.65000,SOLUSDC,107.24000,1.007074
2889,2024-02-17 16:21:01,SOLETH,0.03905,ETHUSDT,2766.50000,SOLUSDT,107.31000,1.006727
2874,2024-02-17 16:20:53,SOLETH,0.03911,ETHBTC,0.05353,SOLBTC,0.00208,1.006373
819,2024-02-17 14:36:33,ETHBTC,0.05373,BTCUSDC,51770.14000,ETHUSDC,2765.91000,1.005676


#Saving All Arbitrage Opportunities

In [19]:
# Here we write the raw data
bid_file = 'bid_arbitrage_oppotunity.csv'
ask_file = 'ask_arbitrage_oppotunity.csv'
# saving the excel
df_bid_arbitrage[df_bid_arbitrage['ratio'] > 1].sort_values(by = 'ratio', ascending = False).to_csv(bid_file, index=False)
df_ask_arbitrage[df_ask_arbitrage['ratio'] < 1].sort_values(by = 'ratio', ascending = True).to_csv(ask_file , index=False)